In [1]:
import pandas as pd
import pathlib
import os 

BASE_PATH = pathlib.Path(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

In [2]:
def eliminar_tildes(texto):
    tildes = {
        "á": "a",
        "é": "e",
        "í": "i",
        "ó": "o",
        "ú": "u",
        "Á": "A",
        "É": "E",
        "Í": "I",
        "Ó": "O",
        "Ú": "U",
    }
    return "".join(tildes.get(c, c) for c in texto)

def airport_name(name):
    name = name.rsplit(" ", 1)[0]
    try:
        right = name.rsplit(" ", 1)[1]
        match right:
            case "International":
                return name.rsplit(" ", 1)[0]
            case _:
                return name
    except:
        return name
    
def percentage(number, total):
    percentage = number / total * 100
    return percentage

## 1. Base de datos de aeropuertos

Creo un dataframe con Pandas. Antes, indico la dirección de la base de origen de datos y donde guardaré la base de datos procesada.
Analizo el contenido de la base de datos.

In [3]:
airport_path = BASE_PATH / ("datasets") / ("custom_datasets") / ("ar-airports.csv")

new_airport_path = BASE_PATH / ("streamlit") / ("data") / ("datasets") / ("ar-airports.csv")

airport_df = pd.read_csv(airport_path)

airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 943 non-null    int64  
 1   ident              943 non-null    object 
 2   type               943 non-null    object 
 3   name               943 non-null    object 
 4   latitude_deg       943 non-null    float64
 5   longitude_deg      943 non-null    float64
 6   elevation_ft       908 non-null    float64
 7   continent          943 non-null    object 
 8   country_name       943 non-null    object 
 9   iso_country        943 non-null    object 
 10  region_name        943 non-null    object 
 11  iso_region         943 non-null    object 
 12  local_region       943 non-null    object 
 13  municipality       927 non-null    object 
 14  scheduled_service  943 non-null    int64  
 15  gps_code           225 non-null    object 
 16  iata_code          103 non

Del análisis de los datos, seleccionamos aquellos campos que brindan información útil al usuario para obtener una única respuesta y que permitan mantener una cantidad aceptable de registros. Luego filtramos la base de datos eliminando todos los registros que tienen al menos un valor nulo en los campos elegidos.

In [4]:
mod_df = airport_df[
    ["type", "name", "elevation_name", "region_name", "municipality", "local_code"]
]
mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   type            943 non-null    object
 1   name            943 non-null    object
 2   elevation_name  908 non-null    object
 3   region_name     943 non-null    object
 4   municipality    927 non-null    object
 5   local_code      761 non-null    object
dtypes: object(6)
memory usage: 44.3+ KB


Analizo la columna type para evitar valores inusuales

In [5]:
mod_df.groupby("type")["name"].count()

type
balloonport         1
closed             41
heliport          144
large_airport       2
medium_airport     68
small_airport     687
Name: name, dtype: int64

Mantenemos los large_airports ya que son conocidos y no incrementan la dificultad. Eliminamos al ballonport

In [6]:
valid_types = ["large_airport", "medium_airport", "small_airport", "heliport"]
mod_df_type = mod_df[mod_df.type.isin(valid_types)]

Elimino todas las filas que contengan valores vacios

In [7]:
mod_df = mod_df.dropna()
mod_df.shape

(755, 6)

Modificacion de las columnas type, name, elevation_name y region_name. Luego, traduzco el nombre de las columnas a español e incluye opciones dentro del nombre

In [8]:
## Type
type_change = {
    "large_airport": "Grande",
    "medium_airport": "Mediano",
    "small_airport": "Pequeño",
    "heliport": "Helipuerto",
}
# Cambio los valores de columna
mod_df = mod_df.replace({"type": type_change}, regex=True)

## Name
# Elimino la ultima palabra de los valores

mod_df["name"] = mod_df["name"].apply(airport_name)

## Elevation name
# Le arreglo el formato
mod_df["elevation_name"] = mod_df["elevation_name"].apply(lambda x: x.title())

## Region_name
# Elimino la palabra Province
mod_df["region_name"] = mod_df["region_name"].replace({" Province": ""}, regex=True)

In [9]:
# Renombro columnas
mod_df.columns = [
    "Tipo (Grande, Mediano, Pequeño, Helipuerto)",
    "Nombre",
    "Elevacion (bajo, medio, altos)",
    "Provincia",
    "Ciudad",
    "Codigo local",
]

mod_df = mod_df.applymap(lambda x: eliminar_tildes(x))
# mod_df["Codigo local"]= mod_df["Codigo local"].apply(lambda x: str(x).title())

mod_df = mod_df.applymap(lambda x: x.title() if isinstance(x, str) else x)
# Creo el nuevo dataset
# mod_df.info()

Guardo la base de datos que será considerada en el juego

In [10]:
mod_df.to_csv(new_airport_path)

## 2. Base de datos de lagos

Creo un dataframe con Pandas. Antes, indico la dirección de la base de origen de datos y donde guardaré la base de datos procesada.
Analizo el contenido de la base de datos.

In [11]:
lakes_path = BASE_PATH / ("datasets") / ("custom_datasets") / ("lagos_arg.csv")

new_lakes_path = BASE_PATH / ("streamlit") / ("data") / ("datasets") / ("lagos_arg.csv")

lakes_df = pd.read_csv(lakes_path)

lakes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Nombre                  52 non-null     object 
 1   Ubicación               52 non-null     object 
 2   Superficie (km²)        52 non-null     int64  
 3   Profundidad máxima (m)  15 non-null     float64
 4   Profundidad media (m)   22 non-null     float64
 5   Latitud                 52 non-null     float64
 6   Longitud                52 non-null     float64
 7   Sup tamaño              52 non-null     object 
dtypes: float64(4), int64(1), object(3)
memory usage: 3.4+ KB


Del análisis de los datos, seleccionamos aquellos campos que brindan información útil al usuario para obtener una única respuesta y que permitan mantener una cantidad aceptable de registros. Luego filtramos la base de datos eliminando todos los registros que tienen al menos un valor nulo en los campos elegidos.

In [12]:
lakes_mod_df = lakes_df[
    ["Nombre", "Ubicación", "Superficie (km²)", "Latitud", "Longitud", "Sup tamaño"]
]
lakes_mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nombre            52 non-null     object 
 1   Ubicación         52 non-null     object 
 2   Superficie (km²)  52 non-null     int64  
 3   Latitud           52 non-null     float64
 4   Longitud          52 non-null     float64
 5   Sup tamaño        52 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.6+ KB


Modificacion de la columna Tamaño

In [13]:
lakes_mod_df["Sup tamaño"] = lakes_mod_df["Sup tamaño"].apply(lambda x: x.title())
lakes_mod_df = lakes_mod_df.applymap(lambda x: x.title() if isinstance(x, str) else x)

C:\Users\franc\AppData\Local\Temp\ipykernel_12028\3887627431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lakes_mod_df["Sup tamaño"] = lakes_mod_df["Sup tamaño"].apply(lambda x: x.title())


Guardo la base de datos que será considerada en el juego

In [14]:
# Elimina todos los tildes
# lakes_mod_df= lakes_mod_df.applymap(lambda x: eliminar_tildes(x))
for col in lakes_mod_df.select_dtypes(include=["object"]).columns:
    lakes_mod_df[col] = lakes_mod_df[col].apply(eliminar_tildes)
lakes_mod_df.to_csv(new_lakes_path)

## 3. Base de datos de conectividad

In [15]:
conect_path = BASE_PATH / ("datasets") / ("custom_datasets") / ("Conectividad_Internet.csv")

new_conect_path = BASE_PATH / ("streamlit") / ("data") / ("datasets") / ("Conectividad_Internet.csv")

conect_df = pd.read_csv(conect_path)

conect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Provincia           4312 non-null   object 
 1   Partido             4312 non-null   object 
 2   Localidad           4312 non-null   object 
 3   Poblacion           4312 non-null   int64  
 4   ADSL                4312 non-null   object 
 5   CABLEMODEM          4312 non-null   object 
 6   DIALUP              4312 non-null   object 
 7   FIBRAOPTICA         4312 non-null   object 
 8   SATELITAL           4312 non-null   object 
 9   WIRELESS            4312 non-null   object 
 10  TELEFONIAFIJA       4312 non-null   object 
 11  3G                  4312 non-null   object 
 12  4G                  4312 non-null   object 
 13  link                3836 non-null   float64
 14  Latitud             4312 non-null   float64
 15  Longitud            4312 non-null   float64
 16  posee_

Visualizo el contenido de la base de datos

In [16]:
conect_df.head(5)

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,link,Latitud,Longitud,posee_conectividad
0,BUENOS AIRES,25 de Mayo,25 de Mayo,23408,SI,SI,NO,SI,SI,SI,SI,SI,SI,6854100.0,-35.433939,-60.173121,SI
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,280,NO,NO,NO,NO,NO,NO,SI,NO,NO,6854010.0,-35.739217,-60.559063,SI
2,BUENOS AIRES,25 de Mayo,Del Valle,899,SI,NO,NO,NO,SI,NO,NO,SI,SI,6854020.0,-35.897345,-60.731609,SI
3,BUENOS AIRES,25 de Mayo,Ernestina,145,SI,NO,NO,NO,NO,NO,NO,NO,NO,6854030.0,-35.270234,-59.557503,SI
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,547,NO,NO,NO,NO,NO,SI,NO,NO,NO,6854040.0,-35.164397,-60.081358,SI


Elimino el campo "Posee conectividad", que ya resume campos anteriores y sería una respuesta muy fácil dadas las pistas obtenidas.
También elimino el link ya que tiene valores dificiles de conocer y reduciría el resultado a suerte.

In [17]:
conect_df = conect_df.rename(
    columns={
        "CABLEMODEM": "Cable-Modem",
        "DIALUP": "Dialup",
        "FIBRAOPTICA": "Fibra Optica",
        "SATELITAL": "Satelital",
        "WIRELESS": "Wireless",
        "TELEFONIAFIJA": "Telefonia Fija",
    }
)

conect_mod_df = conect_df[
    [
        "Provincia",
        "Partido",
        "Localidad",
        "Poblacion",
        "ADSL",
        "Cable-Modem",
        "Dialup",
        "Fibra Optica",
        "Satelital",
        "Wireless",
        "Telefonia Fija",
        "3G",
        "4G",
    ]
]
conect_mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4312 entries, 0 to 4311
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Provincia       4312 non-null   object
 1   Partido         4312 non-null   object
 2   Localidad       4312 non-null   object
 3   Poblacion       4312 non-null   int64 
 4   ADSL            4312 non-null   object
 5   Cable-Modem     4312 non-null   object
 6   Dialup          4312 non-null   object
 7   Fibra Optica    4312 non-null   object
 8   Satelital       4312 non-null   object
 9   Wireless        4312 non-null   object
 10  Telefonia Fija  4312 non-null   object
 11  3G              4312 non-null   object
 12  4G              4312 non-null   object
dtypes: int64(1), object(12)
memory usage: 438.1+ KB


Modificaciones del campo Provincia, reemplazando las mayúsculas por la utilización de mayúscula solo en la primera letra.

In [18]:
conect_mod_df["Provincia"] = conect_mod_df["Provincia"].apply(lambda x: x.title())
conect_mod_df.head(1)
# Le aplico title() y elimino tildes en todo el DataSet
conect_mod_df = conect_mod_df.applymap(lambda x: x.title() if isinstance(x, str) else x)
for col in conect_mod_df.select_dtypes(include=["object"]).columns:
    conect_mod_df[col] = conect_mod_df[col].apply(eliminar_tildes)

C:\Users\franc\AppData\Local\Temp\ipykernel_12028\1037015866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conect_mod_df["Provincia"] = conect_mod_df["Provincia"].apply(lambda x: x.title())


Guardo el nuevo dataset

In [19]:
conect_mod_df.to_csv(new_conect_path)

## 4. Base de datos de censo

Creo un dataframe con Pandas. Antes, indico la dirección de la base de origen de datos y donde guardaré la base de datos procesada.
Analizo el contenido de la base de datos.

In [20]:
censo_path = BASE_PATH / ("datasets") / ("custom_datasets") / ("c2022_tp_c_resumen_adaptado.csv")

new_censo_path = BASE_PATH / ("streamlit") / ("data") / ("datasets") / ("c2022_tp_c_resumen_adaptado.csv")


censo_df = pd.read_csv(censo_path)

censo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Jurisdicción                                   25 non-null     object 
 1   Total de población                             25 non-null     int64  
 2   Población en viviendas particulares            25 non-null     int64  
 3   Población en viviendas colectivas (¹)          25 non-null     int64  
 4   Población en situación de calle(²)             25 non-null     int64  
 5   Varones Total de población                     25 non-null     int64  
 6   Varones Población en viviendas particulares    25 non-null     int64  
 7   Varones Población en viviendas colectivas (¹)  25 non-null     int64  
 8   VaronesPoblación en situación de calle(²)      25 non-null     int64  
 9   Mujeres Total de población                     25 non-nu

Visualizo el contenido de la base de datos

In [21]:
censo_df.head(5)

,Jurisdicción,Total de población,Población en viviendas particulares,Población en viviendas colectivas (¹),Población en situación de calle(²),Varones Total de población,Varones Población en viviendas particulares,Varones Población en viviendas colectivas (¹),VaronesPoblación en situación de calle(²),Mujeres Total de población,Mujeres Población en viviendas particulares,Mujeres Población en viviendas colectivas (¹),Mujeres Población en situación de calle(²),Porcentaje de población en situación de calle
0,Total del país,45892285,45618787,267793,5705,22186791,22010881,171436,4474,23705494,23607906,96357,1231,0.0124
1,Ciudad Autónoma de Buenos Aires,3121707,3095454,23850,2403,1439873,1428781,9216,1876,1681834,1666673,14634,527,0.0770
2,Buenos Aires,17523996,17408906,113664,1426,8470569,8393419,76030,1120,9053427,9015487,37634,306,0.0081
3,Catamarca,429562,427625,1909,28,210464,209088,1365,11,219098,218537,544,17,0.0065
4,Chaco,1129606,1124603,4898,105,547655,543912,3652,91,581951,580691,1246,14,0.0093


Agrego campos conteniendo porcentajes en lugar de números absolutos

In [22]:
censo_df = censo_df.rename(
    columns={
        "Total de población": "Población",
        "Varones Total de población": "Varones",
        "Mujeres Total de población": "Mujeres",
        "Población en situación de calle(²)": "Sit_Calle",
        "Población en viviendas colectivas (¹)": "Viviendas_colectivas",
    }
)

censo_df["Varones Porcentaje"] = censo_df.apply(
    lambda x: int(percentage(x.Varones, x.Población)), axis=1
)
censo_df["Mujeres Porcentaje"] = censo_df.apply(
    lambda x: int(percentage(x.Mujeres, x.Población)), axis=1
)
censo_df["Población en viviendas colectivas Porcentaje"] = censo_df.apply(
    lambda x: round(percentage(x.Viviendas_colectivas, x.Población), 2), axis=1
)
censo_df["Población en situación de calle Porcentaje"] = censo_df.apply(
    lambda x: round(percentage(x.Sit_Calle, x.Población), 2), axis=1
)

censo_df.head(5)

,Jurisdicción,Población,Población en viviendas particulares,Viviendas_colectivas,Sit_Calle,Varones,Varones Población en viviendas particulares,Varones Población en viviendas colectivas (¹),VaronesPoblación en situación de calle(²),Mujeres,Mujeres Población en viviendas particulares,Mujeres Población en viviendas colectivas (¹),Mujeres Población en situación de calle(²),Porcentaje de población en situación de calle,Varones Porcentaje,Mujeres Porcentaje,Población en viviendas colectivas Porcentaje,Población en situación de calle Porcentaje
0,Total del país,45892285,45618787,267793,5705,22186791,22010881,171436,4474,23705494,23607906,96357,1231,0.0124,48,51,0.58,0.01
1,Ciudad Autónoma de Buenos Aires,3121707,3095454,23850,2403,1439873,1428781,9216,1876,1681834,1666673,14634,527,0.0770,46,53,0.76,0.08
2,Buenos Aires,17523996,17408906,113664,1426,8470569,8393419,76030,1120,9053427,9015487,37634,306,0.0081,48,51,0.65,0.01
3,Catamarca,429562,427625,1909,28,210464,209088,1365,11,219098,218537,544,17,0.0065,48,51,0.44,0.01
4,Chaco,1129606,1124603,4898,105,547655,543912,3652,91,581951,580691,1246,14,0.0093,48,51,0.43,0.01


Selecciono las columnas que usaré:

In [23]:
censo_mod_df = censo_df[
    [
        "Jurisdicción",
        "Población",
        "Varones Porcentaje",
        "Mujeres Porcentaje",
        "Población en viviendas colectivas Porcentaje",
        "Población en situación de calle Porcentaje",
    ]
]
censo_mod_df.head(5)

,Jurisdicción,Población,Varones Porcentaje,Mujeres Porcentaje,Población en viviendas colectivas Porcentaje,Población en situación de calle Porcentaje
0,Total del país,45892285,48,51,0.58,0.01
1,Ciudad Autónoma de Buenos Aires,3121707,46,53,0.76,0.08
2,Buenos Aires,17523996,48,51,0.65,0.01
3,Catamarca,429562,48,51,0.44,0.01
4,Chaco,1129606,48,51,0.43,0.01


Elimino Fila "Total del Pais"

In [24]:
censo_mod_df = censo_mod_df.drop(0)

Modificacion Especifica Tierra del Fuego

In [25]:
censo_mod_df["Jurisdicción"] = censo_mod_df["Jurisdicción"].replace(
    {"Tierra del Fuego, Antártida e Islas del Atlántico Sur": "Tierra del Fuego"}
)
censo_mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 1 to 24
Data columns (total 6 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Jurisdicción                                  24 non-null     object 
 1   Población                                     24 non-null     int64  
 2   Varones Porcentaje                            24 non-null     int64  
 3   Mujeres Porcentaje                            24 non-null     int64  
 4   Población en viviendas colectivas Porcentaje  24 non-null     float64
 5   Población en situación de calle Porcentaje    24 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.3+ KB


Creo el nuevo dataset

In [26]:
censo_mod_df = censo_mod_df.applymap(lambda x: x.title() if isinstance(x, str) else x)
for col in censo_mod_df.select_dtypes(include=["object"]).columns:
    censo_mod_df[col] = censo_mod_df[col].apply(eliminar_tildes)
censo_mod_df.to_csv(new_censo_path)